======FairFed============

-> Init Models for clients

-> Get statistics of the dataset

-> start training

      -> for each round-> assigning agg global model to all clients

-> for each client

        > Update its weights based on local and global fairness
        
        > store it
-> SecAgg applied on the server side

In [12]:
import pandas as pd
import time
import numpy as np
import copy
import glob
import sys, os
import torch
import torch.nn as nn
import torch.utils.data as data_utils
from sklearn.metrics import precision_score, recall_score
import pickle
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import os
from scipy.stats import multivariate_normal
import torch, random, copy, os
from collections import OrderedDict
import shutil

In [13]:
# TPR
# Gender: 
# 1: Male, 0: Female
# Fairness: Male- Female
# eq-4: Global- TPR

# data preparation

In [14]:
import torch

if torch.cuda.is_available():
    print("CUDA is available.")
else:
    # CUDA is not available
    print("CUDA is not available. Running on CPU.")


CUDA is available.


In [15]:
 
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        # 9-> we have 9 columns in data 
        self.layer1 = nn.Linear(14, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x



# assigning Models

In [16]:
class AssignModel:
    def __init__(self, global_model_base_path, selected_clients,algo="FF"):      
        self.global_model_path = f"{global_model_base_path}/global_model_{algo}.pt"
        self.selected_clients = selected_clients

    def save_global_model(self, model):
        torch.save(model.state_dict(), self.global_model_path)

    def save_client_models(self, model):
        for client_id in self.selected_clients:
            client_model_path = f"/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/client_{client_id}_model.pth"
            torch.save(model.state_dict(), client_model_path)

In [17]:
class Client:
    def __init__(self, batch_size):
        self.client_id: int = None
        self.valset: DataLoader = None
        self.trainset: DataLoader = None
        self.testset: DataLoader = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")         
        self.model = DeepNet().to(self.device)
        self.learning_rate=0.001
        self.criterion = torch.nn.BCELoss()
        self.optimizer: torch.optim.Optimizer = torch.optim.Adam(
            self.model.parameters(), lr=self.learning_rate
        )
        
        self.Batch = batch_size

        self.fainess_score=0
        self.local_epochs=5
        self.client_df = pd.DataFrame()
        self.selected_clients=[0,1,2,3]

        self.client_fairness=0
        self.global_fairness=0

        
    def get_client_local_dataset(self):
                
        temp_path_data="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/data_fairFed"
        
        with open(temp_path_data+"/clients_training.pkl", "rb") as f:
            self.trainset = pickle.load(f)
            
        with open(temp_path_data+"/clients_validation.pkl", "rb") as f:
            self.valset = pickle.load(f)
         
        with open(temp_path_data+"/clients_testing_wrong.pkl", "rb") as f:
            self.testset  = pickle.load(f)

        self.trainset = self.trainset[self.client_id]
        self.valset = self.valset[self.client_id]       
        self.testset = self.testset[self.client_id]

    def get_stats(self):
        # 1 male 0 female
        total_pr_Y1_A0 = 0
        total_pr_Y1_A1 = 0
        len_of_entire_data=0
        
        for client_id in self.selected_clients:
            self.client_id=client_id
            self.get_client_local_dataset()
            len_of_entire_data+=len(self.trainset.dataset)
            for inputs, labels, A in self.trainset:
                pr_Y1_A1 = torch.sum((labels == 1) & (A == 1)).item()
                pr_Y1_A0 = torch.sum((labels == 1) & (A == 0)).item()
                # len_of_entire_data+=len(self.trainset)
                total_pr_Y1_A1 += pr_Y1_A1
                total_pr_Y1_A0 += pr_Y1_A0
                
            # print("len_of_entire_data::",len_of_entire_data)
            # print("pr_Y1_A1 and pr_Y1_A0",total_pr_Y1_A1,"::",total_pr_Y1_A0)
            # print(total_pr_Y1_A0/len(self.trainset)," , ",print(total_pr_Y1_A1/len(self.trainset)))        
        
        return [total_pr_Y1_A1/len_of_entire_data,total_pr_Y1_A0/len_of_entire_data]
        
    def calculate_fairness_client(self,client_id,y_hat, A, Y,server_acc,client_acc,len_client_data):
        # Calculate counts using torch.sum
        y_hat, A, Y = y_hat.to(self.device), A.to(self.device), Y.to(self.device)

        # 1: Male, 0: Female
        predict_Y_male = torch.sum((y_hat[(A == 1) & (Y == 1)] == 1)).item()
        predict_Y_female = torch.sum((y_hat[(A == 0) & (Y == 1)] == 1)).item()
        
        count_Y_male = torch.sum((Y[(A == 1) & (Y == 1)] == 1)).item()
        count_Y_female = torch.sum((Y[(A == 0) & (Y == 1)] == 1)).item()
    
        # Calculate probabilities
  
        if count_Y_male > 0 and count_Y_female > 0:
            prob_Y_male = predict_Y_male / count_Y_male
            prob_Y_female = predict_Y_female / count_Y_female
            fairness_score = prob_Y_male - prob_Y_female
            
        else:
            # Cal sever_acc from client using Eq.
            total_len_data=sum(len_client_data)   
            temp_server_acc=0
            # get global acc
            for i in range(len(self.selected_clients)):
                temp_server_acc+=client_acc[i]*(len_client_data[i]/total_len_data)
                
            fairness_score= abs(client_acc[client_id]-temp_server_acc)
    
            # print("\n Else fairness_score",fairness_score)
            
        # print(" client fairness_score",fairness_score)
            
        return  fairness_score

    def calculate_ser_fairness(self,client_id,y_hat, A, Y,statistics,server_acc,client_acc,len_client_data):
        # print("calculate_ser_fairness statistics",statistics)
        y_hat, A, Y = y_hat.to(self.device), A.to(self.device), Y.to(self.device)

        # for individual clients!
        #Pr(Ŷ=1|A=1, Y=1)  #male      
        predict_Y_male = torch.sum((y_hat[(A == 1) & (Y == 1)] == 1)).item() #get total
        #Pr(A=0, Y=1)
        count_Y_male = torch.sum((A == 1) & (Y == 1)).item() 
        count_male = torch.sum(A == 1).item()
        

        #Pr(Ŷ=1|A=0, Y=1)  #Female      
        predict_Y_female = torch.sum((y_hat[(A == 0) & (Y == 1)] == 1)).item()  #get total
        #Pr(A=1, Y=1)
        count_Y_female = torch.sum((A == 0) & (Y == 1)).item()
        count_female = torch.sum(A == 0).item()

        
        
        if count_Y_male>0 and count_male>0:
            print("if_male")
            step_2_eq_male=count_Y_male/count_male            
            predict_Y_male=predict_Y_male/count_Y_male #Pr(Ŷ=1|A=0, Y=1)            
            step_3_male=((predict_Y_male * step_2_eq_male)/statistics[0]) 
        else:
            print("else_male")
            step_3_male=0
            
        if count_Y_female>0 and count_female>0 :
            print("if_female")
            step_2_eq_female=count_Y_female/count_female
            predict_Y_female=predict_Y_female/count_Y_female   #Pr(Ŷ=1|A=1, Y=1)
            step_3_female= ((predict_Y_female * step_2_eq_female)/statistics[1])
        else:
            print("else_female")
            step_3_female=0

        temp_fairness_server=step_3_male-step_3_female
            
        # else:
        #     print("else calculate_ser_fairness")
        #     # Cal sever_acc from client using Eq.
        #     total_len_data=sum(len_client_data)   
        #     temp_server_acc=0
        #     for i in range(len(self.selected_clients)):
        #         temp_server_acc+=client_acc[i]*(len_client_data[i]/total_len_data)
                
        #     fairness_score= abs(client_acc[client_id]-temp_server_acc)
            
        #     avg_acc_client=sum(client_acc) /len(client_acc)
        #     temp_fairness_server= abs(avg_acc_client-server_acc)
        #      abs(client_acc[client_id]-temp_server_acc)
        #     # print("Else calculate_ser_fairness",temp_fairness_server)

        # print("calculate_ser_fairness temp_fairness_server",self.client_id,temp_fairness_server)
        return temp_fairness_server

    def get_client_weights(self):
        # self.client_id = client_id
        
        clients_weights=[]
        for client_id in self.selected_clients:
            self.client_id=client_id
            self.get_client_local_dataset()
            clients_weights.append(len(self.trainset.dataset))

        return clients_weights
        
    def train_FA(self, client_id: int, model_path,learning_rate=0.001,num_epochs=5):
        self.model.load_state_dict(torch.load(model_path,map_location=self.device))
        
        self.model.train()
        for epoch in range(num_epochs):
            print("epoch",epoch)
            for x, y,z in self.trainset:
                x, y = x.to(self.device), y.to(self.device)
                logits = self.model(x)
                loss = self.criterion(logits, y)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
        models_directory = "/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/"
        model_path_1 = os.path.join(models_directory, f"client_{client_id}_model.pth")          
        torch.save(self.model.state_dict(), model_path_1)
        print(f"client_{client_id} is updated \n\n\n" )
                
        return list(self.model.state_dict().values()), len(self.trainset.dataset)
       
        
    def train_FF(self, client_id: int, model_path,statistics,server_acc,clients_acc,len_client_data, num_epochs=5, learning_rate=0.001):
        self.client_id = client_id
        self.get_client_local_dataset()
        # print("len_of_entire_data FF",len(self.trainset))
        # Define loss function and optimizer
        
        self.fainess_score=0
      
        self.model.load_state_dict(torch.load(model_path,map_location=self.device))
        
        self.model.train()
        for epoch in range(num_epochs):
            # Set the model to training mode

            for inputs, labels, sens in self.trainset:
                
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs).to(self.device)
                loss = self.criterion(outputs, labels.float())
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

            # Calculate and print the training accuracy for this epoch (optional)
            
            correct = 0
            total = 0
            size = 0
            loss = 0
            correct = 0
            total = 0
            predicted_labels = []
            true_labels = []
            final_fairness=[]
            final_fairness_server=[]
            
            with torch.no_grad():
                print("======Validation========")
                for inputs, labels,sens in self.valset:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    
                    outputs = self.model(inputs)
                    loss += self.criterion(outputs, labels)
                    predicted = outputs > 0.5

                    predicted_labels.extend(predicted.cpu().numpy())
                    true_labels.extend(labels.cpu().numpy())                   
                    
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()                   

            loss = loss / len(self.valset)        
            acc = correct/total
 
            fairNess_per_batch=self.calculate_fairness_client(self.client_id,torch.round(outputs).squeeze(), sens.squeeze(), labels.squeeze(),
                                                       server_acc,
                                                       clients_acc,
                                                       len_client_data)
            final_fairness.append(fairNess_per_batch)
            self.fainess_score=np.mean(final_fairness)

            #this is needed for Eq.7: global fairness
            fairNess_per_batch_server=self.calculate_ser_fairness(self.client_id,torch.round(outputs).squeeze(), sens.squeeze(), labels.squeeze(),
                                                                  statistics,
                                                                  server_acc,
                                                                  clients_acc,
                                                                  len_client_data)
            final_fairness_server.append(fairNess_per_batch_server)            
            ser_score=np.mean(final_fairness_server)
                        
            precision = precision_score(true_labels, predicted_labels,zero_division=0.0)            
            recall = recall_score(true_labels, predicted_labels)
            
            # accuracy = 100 * correct / total
            print(f"Epoch {epoch+1}/{num_epochs}, Val Accuracy: {acc:.5f}")
            print(f"Epoch {epoch+1}/{num_epochs}, Val Fairness: {self.fainess_score:.5f}")
            print(f"Epoch {epoch+1}/{num_epochs}, Val Recall: {recall:.5f}")


        # Optionally, save the trained model parameters
        #store model client model
        models_directory = "/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/"
        model_path_1 = os.path.join(models_directory, f"client_{client_id}_model.pth")          
        torch.save(self.model.state_dict(), model_path_1)
        print(f"client_{client_id} is updated and its fairness score wrt global_model {ser_score} \n\n\n" )

        
        return list(self.model.state_dict().values()), len(self.trainset.dataset),self.model.state_dict(), self.fainess_score, ser_score

    @torch.no_grad()
    def client_evaluate(self, val=False):
        
        size = 0
        loss = 0
        correct = 0
        total = 0
        predicted_labels = []
        true_labels = []
        len_of_data=[]
        all_client_acc=[]
        
        models_directory = "/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/"
                 
        for client_id in self.selected_clients:
            
            model_path = os.path.join(models_directory, f"client_{client_id}_model.pth")            
        
            self.model.load_state_dict(torch.load(model_path,map_location=self.device))

            self.model.eval()

            for inputs, targets,sens in self.testset:
                
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                
                outputs = self.model(inputs)
                loss += self.criterion(outputs, targets)
                predicted = outputs > 0.5
                
                predicted_labels.extend(predicted.cpu().numpy())
                true_labels.extend(targets.cpu().numpy())
                
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
                
                len_of_data.append(len(self.testset))
            
            loss = loss / len(self.testset)
            acc = correct / total
            all_client_acc.append(acc)
                        
            precision = precision_score(true_labels, predicted_labels,zero_division=0.0)
            
            recall = recall_score(true_labels, predicted_labels)
                            
        return loss, acc, precision, recall,len_of_data, all_client_acc

    
    @torch.no_grad()
    def server_evaluate(self,path,statistics,all_client_acc,len_of_data):

        # print("Global Model testing Starts")
        # print("kindly check the Path. Select it based on FedAvg Model")
        # print("warning: Test data has already been used")
        temp_path_data="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/"

        with open(temp_path_data+"/testing_client.pkl", "rb") as f:
            testset  = pickle.load(f)
            
        
        # path="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/global_model_2.pt"
        
        self.model.load_state_dict(torch.load(path,map_location=self.device))
        self.model.eval()
        size = 0
        loss = 0
        correct = 0
        total = 0
        predicted_labels = []
        true_labels = []
        final_fairness=[]

        self.testset=testset[1]
        for inputs, targets,sens in self.testset:
            
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            
            outputs = self.model(inputs)
            loss += self.criterion(outputs, targets)
            predicted = outputs > 0.5

            
#            print("predicted",predicted)
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(targets.cpu().numpy())
            
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
        
        loss = loss / len(self.testset)
        ser_acc = correct / total
        
        fairNess_per_batch=self.calculate_ser_fairness(self.client_id,torch.round(outputs).squeeze(), sens.squeeze(),
                                                                   targets.squeeze(),
                                                                  statistics,
                                                                  ser_acc,
                                                                  all_client_acc,
                                                                  len_of_data)
        
        # fairNess_per_batch=self.calculate_ser_fairness(torch.round(outputs).squeeze(), sens.squeeze(), targets.squeeze(),
        #                                             ser_acc,
        #                                             all_client_acc,
        #                                             len_of_data)
        final_fairness.append(fairNess_per_batch)
        fairness_global=np.mean(final_fairness)

        # print("loss: %f\n" % (loss))
        
        # print(f"Global Testing Accuracy: {ser_acc:.5%}")
        # print(f" Global Fairness: {fairness_global:.5f}%")
        precision = precision_score(true_labels, predicted_labels,zero_division=0.0)
        
        recall = recall_score(true_labels, predicted_labels)
        
        # print(f"Global Precision: {precision:.5%}")
        # print(f"Global Recall: {recall:.5%}")
        
        return loss, ser_acc, precision, recall,fairness_global
                                            

In [21]:
class Serverbase:
    def __init__ (self,model):
        self.model = model
        self.global_model=model
        self.num_rounds=2
        self.local_epoch=2
        self.optimizer=2
        self.lr=0.001
        self.beta=1
        self.batch_size=32
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    
        self.updated_params_cache = []
        self.weights_cache = []
        self.model_dict = []
        
        self.global_params_dict: OrderedDict[str : torch.Tensor] = None

        self.backbone=DeepNet
        _dummy_model = self.backbone()
        self.global_params_dict = OrderedDict(_dummy_model.state_dict())

        self.temp_dir="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/"

        self.fair_global_t_step=0
        self.fair_local_client=0   
        
        self.acc_clients=[]
        self.clients_id=[]
        self.precision_clients=[]
        self.recall_clients=[]
        
        self.global_acc=0
        self.fairness_diff=[]
        self.clients_weights=[]
        self.final_agg_weights_clients=[]
        self.selected_clients=[0,1,2,3]
        self.statistics=0
        self.clients_weights=[]

    def global_fairness(self,weights, temp_ser_fairness): 
        # all_clients_fairness is based on equation: 7
        # upper part is done in client class
        # self.statistics cal. pr(y=1,a=0)
        
        # print("global_fairness::",all_clients_fairness)
        # print("weights::",weights)
        
        weight_sum = sum(weights)  
        final_global_score=0
        
        for i in range(len(temp_ser_fairness)):
            temp=weights[i]/weight_sum            
            final_global_score+=temp*temp_ser_fairness[i]
            
        self.fair_global=final_global_score

    
        
    def fedAlgo(self, num_rounds = 2, local_epochs = 2, learning_rate = 0.001, beta = 1, optimizer = 'adam',algo="FF"):
        
        client = Client(batch_size=64)
        path=f"/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/global_model_{algo}.pt"
        models_directory = "/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/"
        
        #before we start the training, we need statistics
        print("Getting data statistics....")
        self.statistics= client.get_stats()      
        
        print("self.statistics",self.statistics)
        self.weights_cache=client.get_client_weights()

        print("self.weights_cache:",self.weights_cache)
        for round_ in tqdm(range(num_rounds)):
  
            #each round we need empty lists
            self.updated_params_cache = []
            # self.weights_cache = []
            self.model_dict=[]
            self.clients_weights=[]
            self.final_agg_weights_clients=[]
            self.fair_global=0
            self.fair_local_all_client=[]
            temp_ser_fairness=[]

            print ("round::",round_)
            
             # clients and server acc for calculating Fairness whenever fairness_client is undefined
        
            loss_client, acc_client,precision, recall,len_of_data,all_client_acc = client.client_evaluate() 

            # to ge the acc of global model 
            loss_client, accuracy_server, pre_server, recall_ser,fairness_global = client.server_evaluate(path,self.statistics,all_client_acc,len_of_data)

            
            for client_id in self.selected_clients:
                self.global_acc=0
                
                
                # weight is length of dataset
                
                # Clone the model from server and assign it back to the clients
                model_path=os.path.join(self.temp_dir, f"global_model_{algo}.pt") 

                if algo=="FF":
                    
                    updated_params, weight, model_dict_list,fairness_client,ser_fairness = client.train_FF(client_id, 
                                                                                                             model_path, 
                                                                                                             statistics=self.statistics,
                                                                                                             server_acc=accuracy_server,
                                                                                                             clients_acc=all_client_acc,
                                                                                                             len_client_data=len_of_data,
                                                                                                             num_epochs=local_epochs, 
                                                                                                             learning_rate=learning_rate)        
                                
                    print("updated_params FF",type(updated_params[0]))
                    self.updated_params_cache.append(updated_params)
                    #not sure how to pass updated weights to next round
                    # self.weights_cache.append(weight)
                    self.model_dict.append(model_dict_list)
                    temp_ser_fairness.append(ser_fairness)
                    
                    
                    # self.global_acc=acc_server
                    self.fair_local_client=fairness_client
                    self.fair_local_all_client.append(fairness_client)
                
                if algo=="FA":
                    updated_params, weight, = client.train_FA(client_id, model_path,
                                                              num_epochs=local_epochs,
                                                              learning_rate=learning_rate)
                                                                                            
                                
                    # store it for SecAgg
                    self.updated_params_cache.append(updated_params)
                    self.weights_cache.append(weight)
                    

            print("temp_ser_fairness::",temp_ser_fairness)
            print("global self.weights_cache::",self.weights_cache)
            self.global_fairness(self.weights_cache, temp_ser_fairness)

            print("for round", round_,", global fairness, accuracy, and recall are: ",self.fair_global,", ",
                                                                                  accuracy_server,", ",
                                                                                  recall_ser,"\n\n\n")
            
            if algo=="FF":
                # agg all the weights on serverside
                # Update the weights of each client
                self.aggregate_client_weight_FedFair_FedAVG(self.updated_params_cache,self.model_dict, self.weights_cache,self.acc_clients,
                                                            self.global_acc,self.fair_local_client,
                                                             self.fair_global, self.fair_local_all_client)
                
                # self.aggregate_client_weight_FedFair_part_1(self.model_dict, self.weights_cache,self.acc_clients,
                #                                             self.global_acc,self.fair_local_client,
                #                                              self.fair_global, self.fair_local_all_client)
                # self.client_weights_update_part_2(self.clients_weights)
                # self.aggregate_parameters_FedAvg_updated(self.final_agg_weights_clients)
                
            elif algo=="FA":
                self.aggregate_parameters_FedAvg(self.updated_params_cache,self.weights_cache)
            else:
                print("--Choose Algo --")
                break
        
            
            torch.save(self.global_model.state_dict(), os.path.join(self.temp_dir, f"global_model_{algo}.pt"),)

            

    @torch.no_grad()
    def aggregate_parameters_FedAvg(self, updated_params_cache, weights_cache):
        weight_sum = sum(weights_cache)  
        
        weights = torch.tensor(weights_cache, device=self.device) / weight_sum
        
        aggregated_params = []

        for params in zip(*updated_params_cache):
            aggregated_params.append(
                torch.sum(weights * torch.stack(params, dim=-1), dim=-1)
            )
        
        aggregated_model_params = {}
        global_state_dict_keys = list(self.global_model.state_dict().keys())
        
        for param_name, param_value in zip(global_state_dict_keys, aggregated_params):
            aggregated_model_params[param_name] = param_value


        # aggregated_model_params = {param_name: param_value for param_name, param_value in zip(self.global_model.state_dict().keys(), aggregated_params)}

        self.global_model.load_state_dict(aggregated_model_params)


    @torch.no_grad()
    def aggregate_client_weight_FedFair_FedAVG (self,updated_params_cache, model_dict_list ,weights_cache, acc_clients, global_acc,fairness_client,fairness_global,fairness_all_client):

        mean_fairness=np.mean(fairness_all_client)
        
        # Delta_t_C_i = abs(self.fair_global - self.fair_local_client) 
        
        temp=weights_cache
        temp_list=[]
        
        # print("client fairness",fairness_all_client)
        print("fair_global",self.fair_global)
        print("weights_cache:: ",weights_cache)
        for i in range(len(temp)):
             Delta_t_C_i=abs(self.fair_global-fairness_all_client[i])
             self.weights_cache[i]=temp[i]-(self.beta*(Delta_t_C_i-mean_fairness))
             print("self.weights_cache[i] ",self.weights_cache[i])

        weights_cache=self.weights_cache
        weight_sum = sum(weights_cache)  
        
        weights = torch.tensor(weights_cache, device=self.device) / weight_sum
        
        aggregated_params = []

        for params in zip(*updated_params_cache):
            aggregated_params.append(
                torch.sum(weights * torch.stack(params, dim=-1), dim=-1)
            )
        
        aggregated_model_params = {}
        global_state_dict_keys = list(self.global_model.state_dict().keys())
        
        for param_name, param_value in zip(global_state_dict_keys, aggregated_params):
            aggregated_model_params[param_name] = param_value

  
    
    @torch.no_grad()
    def aggregate_parameters_FedAvg_updated(self, model_dict_list):        
        w_avg = copy.deepcopy(model_dict_list[0])
        for k in w_avg.keys():
            for i in range(1, len(model_dict_list)):
                w_avg[k] += model_dict_list[i][k]
            w_avg[k] = torch.div(w_avg[k], len(model_dict_list))
    
        self.global_model.load_state_dict(w_avg)

    
    @torch.no_grad()
    def aggregate_client_weight_FedFair_part_1 (self, model_dict_list ,weights_cache, acc_clients, global_acc,fairness_client,fairness_global,fairness_all_client):

        
        Delta_t_C_i = abs(self.fair_global - self.fair_local_client) 
        
        
        for client_id in range(len(self.selected_clients)):
        
            w_avg = copy.deepcopy(model_dict_list[client_id])
            temp = copy.deepcopy(model_dict_list[client_id])
            
            for k in w_avg.keys():
                for i in range(1, len(model_dict_list)):
                    w_avg[k] += model_dict_list[i][k]
                w_avg[k] = torch.div(w_avg[k], len(model_dict_list))
                
                w_avg[k]=temp[k]-(self.beta*(Delta_t_C_i-w_avg[k]))
                
            self.clients_weights.append(w_avg)

            
    def client_weights_update_part_2(self,agg_weights):
        # print("Before agg_weights:: ", agg_weights[0]['layer1.weight'])     .
        # total_weights=0
        # for model in agg_weights:
        #     for param in model.values():
        #         total_weights += param.sum().item()
        
        # print("\n total_weights",total_weights)   
        
        temp=copy.deepcopy(agg_weights)
        sum_params = temp[0]
        
        for model in temp[1:]:
            for name, param in model.items():
                
                if name in sum_params:
                    sum_params[name] += param
                    
        total_sum = sum(torch.sum(param) for param in sum_params.values())  
        print("\ntotal_sum",total_sum)
        
        for model in agg_weights:
             for name in model.keys():
                 # print("name",name,"\n")
                 # print("\n model",model[name])
                 # print("\n sum_params[name]",sum_params[name])               
                 # print("\n model[name]  /= sum_params[name]",model[name]  /total_sum)
                 
                 # model[name]  /= sum_params[name]
                 model[name]  /= total_sum
             # print("type of mdoel", model)
             self.final_agg_weights_clients.append(model)     

In [23]:
# FairFed "FF" and FedAVG "FA"

algo="FF" 

if algo=="FA":
    print("Comment out part1 and part2 def_fairFed ")
    
else:
    print("Algorithm is ", algo)
    
global_model_base_path = "/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models"
shutil.rmtree(global_model_base_path)
os.mkdir(global_model_base_path)

selected_clients = [0, 1, 2, 3]

assigner = AssignModel(global_model_base_path, selected_clients,algo)

temp_model = DeepNet()  # Replace with your model instantiation
assigner.save_global_model(temp_model)
assigner.save_client_models(temp_model)


model = DeepNet()
server = Serverbase(model)
server.fedAlgo(num_rounds=10,local_epochs=10,learning_rate = 0.001, beta = 1,algo=algo)  

Algorithm is  FF
Getting data statistics....
self.statistics [0.21462357114702404, 0.1967327990189638]
self.weights_cache: [27424, 28357, 40171, 41046]


  0%|                                                                  | 0/10 [00:00<?, ?it/s]

round:: 0
else_male
if_female
======Validation========
if_male
else_female
Epoch 1/10, Val Accuracy: 0.77424
Epoch 1/10, Val Fairness: 0.61406
Epoch 1/10, Val Recall: 0.67559
======Validation========
if_male
else_female
Epoch 2/10, Val Accuracy: 0.77792
Epoch 2/10, Val Fairness: 0.61406
Epoch 2/10, Val Recall: 0.68351
======Validation========
if_male
else_female
Epoch 3/10, Val Accuracy: 0.77965
Epoch 3/10, Val Fairness: 0.61406
Epoch 3/10, Val Recall: 0.81193
======Validation========
if_male
else_female
Epoch 4/10, Val Accuracy: 0.79329
Epoch 4/10, Val Fairness: 0.61406
Epoch 4/10, Val Recall: 0.75411
======Validation========
if_male
else_female
Epoch 5/10, Val Accuracy: 0.79784
Epoch 5/10, Val Fairness: 0.61406
Epoch 5/10, Val Recall: 0.68046
======Validation========
if_male
else_female
Epoch 6/10, Val Accuracy: 0.79870
Epoch 6/10, Val Fairness: 0.61406
Epoch 6/10, Val Recall: 0.71394
======Validation========
if_male
else_female
Epoch 7/10, Val Accuracy: 0.80649
Epoch 7/10, Val Fairn

 10%|█████▊                                                    | 1/10 [01:04<09:44, 64.98s/it]

updated_params FF <class 'torch.Tensor'>
temp_ser_fairness:: [1.9413835322926234, -1.8483769124908929, 1.285329786897185, -5.083036509349955]
global self.weights_cache:: [27424, 28357, 40171, 41046]
for round 0 , global fairness, accuracy, and recall are:  -1.1400112104646984 ,  0.6135610946487712 ,  0.052546560243253514 



fair_global -1.1400112104646984
weights_cache::  [27424, 28357, 40171, 41046]
self.weights_cache[i]  27422.859988789536
self.weights_cache[i]  28355.859988789536
self.weights_cache[i]  40169.85998878953
self.weights_cache[i]  41044.85998878953
round:: 1
else_male
if_female
======Validation========
if_male
else_female
Epoch 1/10, Val Accuracy: 0.76905
Epoch 1/10, Val Fairness: 0.78455
Epoch 1/10, Val Recall: 0.58430
======Validation========
if_male
else_female
Epoch 2/10, Val Accuracy: 0.78312
Epoch 2/10, Val Fairness: 0.78455
Epoch 2/10, Val Recall: 0.74437
======Validation========
if_male
else_female
Epoch 3/10, Val Accuracy: 0.79199
Epoch 3/10, Val Fairness: 0.78

...

else_male
if_female
Epoch 10/10, Val Accuracy: 0.78149
Epoch 10/10, Val Fairness: 0.78656
Epoch 10/10, Val Recall: 0.72313
client_3 is updated and its fairness score wrt global_model -2.5415182546749775 



updated_params FF <class 'torch.Tensor'>
temp_ser_fairness:: [1.9413835322926234, -1.8483769124908929, 1.606662233621481, -2.5415182546749775]
global self.weights_cache:: [27418.848781170364, 28351.830554002296, 40165.82405800404, 41040.82853942056]
for round 7 , global fairness, accuracy, and recall are:  -0.2843351472745419 ,  0.6331183545407008 ,  0.0 



fair_global -0.2843351472745419
weights_cache::  [27418.848781170364, 28351.830554002296, 40165.82405800404, 41040.82853942056]
self.weights_cache[i]  27418.568352986873
self.weights_cache[i]  28351.545740334757
self.weights_cache[i]  40165.53796523698
self.weights_cache[i]  41040.542533449545
round:: 8
if_male
if_female
======Validation========
if_male
else_female
Epoch 1/10, Val Accuracy: 0.76905
Epoch 1/10, Val Fairness: 0.77

 90%|████████████████████████████████████████████████████▏     | 9/10 [09:06<00:58, 58.20s/it]

else_male
if_female
Epoch 10/10, Val Accuracy: 0.78867
Epoch 10/10, Val Fairness: 0.78259
Epoch 10/10, Val Recall: 0.77824
client_3 is updated and its fairness score wrt global_model -2.5415182546749775 



updated_params FF <class 'torch.Tensor'>
temp_ser_fairness:: [2.329660238751148, -1.4787015299927142, 1.285329786897185, -2.5415182546749775]
global self.weights_cache:: [27418.568352986873, 28351.545740334757, 40165.53796523698, 41040.542533449545]
for round 8 , global fairness, accuracy, and recall are:  -0.22432256449405064 ,  0.6331183545407008 ,  0.0 



fair_global -0.22432256449405064
weights_cache::  [27418.568352986873, 28351.545740334757, 40165.53796523698, 41040.542533449545]
self.weights_cache[i]  27418.34818064348
self.weights_cache[i]  28351.320716549635
self.weights_cache[i]  40165.31152530597
self.weights_cache[i]  41040.31687925109
round:: 9
else_male
if_female
======Validation========
if_male
else_female
Epoch 1/10, Val Accuracy: 0.77035
Epoch 1/10, Val Fairness: 0

100%|█████████████████████████████████████████████████████████| 10/10 [10:02<00:00, 60.27s/it]

else_male
if_female
Epoch 10/10, Val Accuracy: 0.78522
Epoch 10/10, Val Fairness: 0.78195
Epoch 10/10, Val Recall: 0.76711
client_3 is updated and its fairness score wrt global_model -2.5415182546749775 



updated_params FF <class 'torch.Tensor'>
temp_ser_fairness:: [2.329660238751148, -1.6635392212418034, 1.285329786897185, -2.5415182546749775]
global self.weights_cache:: [27418.34818064348, 28351.320716549635, 40165.31152530597, 41040.31687925109]
for round 9 , global fairness, accuracy, and recall are:  -0.2625812270431021 ,  0.6331183545407008 ,  0.0 



fair_global -0.2625812270431021
weights_cache::  [27418.34818064348, 28351.320716549635, 40165.31152530597, 41040.31687925109]
self.weights_cache[i]  27418.093079864488
self.weights_cache[i]  28351.05829749414
self.weights_cache[i]  40165.046319640824
self.weights_cache[i]  41040.04927984256


In [ ]:
fair_global -0.920953494483727
weights_cache::  [27424, 28357, 40171, 41046]

In [ ]:
 # client_acc [0.621039359719329, 0.621039359719329, 0.621039359719329, 0.621039359719329]
 # client_acc [0.7801227935533385, 0.7802324306545335, 0.7809998903628989, 0.7817810547089135]

In [ ]:
27424+ 28357+ 40171+ 41046



In [ ]:
import torch

# Create instances of your DeepNet model
model1 = DeepNet()
model2 = DeepNet()
model3 = DeepNet()

# Combine the models into a list
models = [model1, model2, model3]

# Initialize a variable to store the sum of weights
total_weights = 0

# Loop through the models and sum their parameters
for model in models:
    for param in model.parameters():
        total_weights += param.sum().item()

print("Sum of weights across all models:", total_weights)
